In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

In [10]:
import numpy as np
from matplotlib import pyplot as plt

from dl import Model
from dl import Dense
from dl.activations import TanH, ReLU, Sigmoid
from dl.losses import BinaryCrossEntropy
from dl.regularization import Dropout
from dl.optimizers import RMSProp

In [3]:
m, n = 2000, 4
inputs = np.random.uniform(-1, 1, size=(m, n))
outputs = (inputs > 0).astype(np.int64)

In [4]:
# train, val, test split
def train_test_split(inputs, outputs, pourcentage):
    m, _ = inputs.shape
    permutation = np.random.permutation(m)
    m_train = np.ceil(m * pourcentage).astype(int)

    inputs_train = inputs[permutation[:m_train]]
    inputs_test = inputs[permutation[m_train:]]

    outputs_train = outputs[permutation[:m_train]]
    outputs_test = outputs[permutation[m_train:]]

    return inputs_train, outputs_train, inputs_test, outputs_test

inputs_train, outputs_train, inputs_test, outputs_test = train_test_split(inputs, outputs, pourcentage=0.8)
inputs_val, outputs_val, inputs_test, outputs_test = train_test_split(inputs_test, outputs_test, pourcentage=0.8)

print(inputs_train.shape, outputs_train.shape)
print(inputs_test.shape, outputs_test.shape)
print(inputs_val.shape, outputs_val.shape)

(1600, 4) (1600, 4)
(80, 4) (80, 4)
(320, 4) (320, 4)


In [5]:
model = Model([
    Dense(n, 8),
    ReLU(),
    Dense(8, 16),
    ReLU(),
    Dense(16, 32),
    TanH(),
    Dropout(keep_prob=0.8),
    Dense(32, n),
    Sigmoid()
])

In [6]:
loss = BinaryCrossEntropy()

In [16]:
n_epochs = 500
learning_rate = 0.1
batch_size = 128
beta = 0.99

In [17]:
optimizer = RMSProp(learning_rate, batch_size, beta)

In [18]:
model.compile(loss, optimizer)

In [19]:
history = model.optimize(inputs_train, outputs_train, n_epochs, verbose=False, inputs_val=inputs_val, outputs_val=outputs_val)

In [ ]:
X = list(range(len(history["train_losses"])))

fig, axs = plt.subplots(2, sharex=True)
fig.suptitle('Training & Validation Loss')

axs[0].plot(X, history["train_losses"])
axs[0].set_xlabel("Epoch")
axs[0].set_ylabel("Training Loss")

axs[1].plot(X, history["val_losses"])
axs[1].set_xlabel("Epoch")
axs[1].set_ylabel("Validation Loss")

plt.show()

In [22]:
predictions = model(inputs_val, is_optimizing=False)
l = loss.forward(predictions, outputs_val)

print(l)

0.49894159785462644


In [23]:
predictions = model(inputs_test, is_optimizing=False)
l = loss.forward(predictions, outputs_test)

print(l)

0.39229753425338504
